#### Notebook for running React experiments

In [5]:
import logging
import sys, os
sys.path.append('..')
root  = '../root/'

In [6]:
import dotenv
dotenv.load_dotenv()

True

In [7]:
import json
import joblib
from util import summarize_react_trial, log_react_trial, save_agents, save_results
from agents import ReactReflectAgent, ReactAgent, ReflexionStrategy
from llm import AnyOpenAILLM

#### Load the HotpotQA Sample

In [8]:
hotpot = joblib.load('../data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

#### Define the Reflexion Strategy

In [9]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [10]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

#### Define the LLMs

In [11]:
action_llm = "gpt-4"
self_reflect_llm = "gpt-4"

#### Initialize a React Agent for each question

In [12]:
agent_cls = ReactReflectAgent if strategy != ReflexionStrategy.NONE else ReactAgent
if strategy != ReflexionStrategy.NONE:
    agents = [ReactReflectAgent(
                    row['question'],
                    row['answer'],
                 react_llm = AnyOpenAILLM(
                                             temperature=0,
                                             max_tokens=100,
                                             model_name=action_llm,
                                             model_kwargs={"stop": "\n"},
                                             openai_api_key=os.environ['OPENAI_API_KEY']),
                 reflect_llm = AnyOpenAILLM(
                                               temperature=0,
                                               max_tokens=250,
                                               model_name=self_reflect_llm,
                                               openai_api_key=os.environ['OPENAI_API_KEY']),
    ) for _, row in hotpot.iterrows()]
else:
    agents = [ReactAgent(
                    row['question'],
                    row['answer'],
                 react_llm = AnyOpenAILLM(
                                             temperature=0,
                                             max_tokens=100,
                                             model_name=action_llm,
                                             model_kwargs={"stop": "\n"},
                                             openai_api_key=os.environ['OPENAI_API_KEY']),
    ) for _, row in hotpot.iterrows()]

#### Run `n` trials

In [14]:
n = 5
trial = 0
log = ''
results = []

run_dir = os.path.join(root, 'ReAct', strategy.value + '_'+ self_reflect_llm)
os.makedirs(run_dir, exist_ok=True)

# Basic logging config that writes to a file
logging.basicConfig(
    filename=os.path.join(run_dir, f'{n}_questions.log'),
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

In [15]:
for i in range(n):
    for agent in [a for a in agents if not a.is_correct()]:
        if strategy != ReflexionStrategy.NONE:
            agent.run(reflect_strategy = strategy)
        else:
            agent.run()
        logging.info(f'Answer: {agent.key}')
    trial += 1
    log += log_react_trial(agents, trial)
    correct, incorrect, halted = summarize_react_trial(agents)
    results.append({"trial": trial, "correct": len(correct), "incorrect": len(incorrect), "halted": len(halted)})
    save_results(agents, results, run_dir)
    #save_agents(agents, os.path.join(run_dir, 'ReAct', strategy.value, 'agents', f'trial_{trial}'))
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}, Halted: {len(halted)}')

/Users/becklabash/Documents/Research/reflexion/reflexion/env/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/becklabash/Documents/Research/reflexion/reflexion/env/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Finished Trial 1, Correct: 39, Incorrect: 45, Halted: 16
Finished Trial 2, Correct: 47, Incorrect: 30, Halted: 23
Finished Trial 3, Correct: 51, Incorrect: 29, Halted: 20
Finished Trial 4, Correct: 52, Incorrect: 25, Halted: 23
Finished Trial 5, Correct: 53, Incorrect: 26, Halted: 21


#### Save the result log

In [ ]:
with open(os.path.join(run_dir, f'{len(agents)}_questions_{trial}_trials.txt'), 'w') as f:
    f.write(log)
save_agents(agents, os.path.join('ReAct', strategy.value, 'agents'))